In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
insurance_df = pd.read_csv('2017_Iowa_ACA_Compliant_Premiums.csv')
insurance_df.head(2)

,Plan ID,Insurance Company,Plan Name,Rating Area,Age,Metal Level,Market Place,Non-Tobacco Premium,Tobacco Premium,Counties,Rating Area Centroid,County Boundaries of Iowa,US Counties,Iowa Watersheds (HUC 10),Iowa Watershed Sub-Basins (HUC 08),Iowa ZIP Code Tabulation Areas,:@computed_region_4qgw_m3n5
0,72160IA0180002,"Wellmark, Inc.",SimplyBlue 5500 PPO,Rating Area 2,55,Bronze,Off,753.36,866.36,"Dallas, Jasper, Madison, Marion, Polk, Warren","(41.52, -93.54)",64,1878,356,29,483,2
1,27651IA0060005,"Gundersen Health Plan, Inc.",Bronze HSA $5750 - 20%,Rating Area 7,60,Bronze,On,863.59,863.59,"Allamakee, Bremer, Butler, Cerro Gordo, Chicka...","(43.08, -92.99)",19,1781,194,15,284,7


In [3]:
insurance_df.columns

Index(['Plan ID', 'Insurance Company', 'Plan Name', 'Rating Area', 'Age',
       'Metal Level', 'Market Place', 'Non-Tobacco Premium', 'Tobacco Premium',
       'Counties', 'Rating Area Centroid', 'County Boundaries of Iowa',
       'US Counties', 'Iowa Watersheds (HUC 10)',
       'Iowa Watershed Sub-Basins (HUC 08)', 'Iowa ZIP Code Tabulation Areas',
       ':@computed_region_4qgw_m3n5'],
      dtype='object')

In [4]:
insurance_df.nunique()

Plan ID                                 116
Insurance Company                         8
Plan Name                               115
Rating Area                               7
Age                                      46
Metal Level                               5
Market Place                              2
Non-Tobacco Premium                   10650
Tobacco Premium                       10695
Counties                                  7
Rating Area Centroid                      7
County Boundaries of Iowa                 7
US Counties                               7
Iowa Watersheds (HUC 10)                  7
Iowa Watershed Sub-Basins (HUC 08)        7
Iowa ZIP Code Tabulation Areas            7
:@computed_region_4qgw_m3n5               7
dtype: int64

Will delete columns with 100+ categorical values, as well as all but one column (Rating_Area) with 7 values, since those columns are perfectly multicollinear with Rating_Area and do not impact the dependent variable non-Tobacco Premium.

In [5]:
insurance_df = insurance_df.drop(columns=['Plan ID', 'Plan Name', 'Rating Area Centroid', 'County Boundaries of Iowa',
       'US Counties', 'Iowa Watersheds (HUC 10)',
       'Iowa Watershed Sub-Basins (HUC 08)', 'Iowa ZIP Code Tabulation Areas',
       ':@computed_region_4qgw_m3n5'])

In [6]:
insurance_df = insurance_df.rename(columns = {"Rating Area":"Rating_Area", "Insurance Company":"Insurance_Company", \
    "Metal Level": "Metal_Level", "Market Place": "Marketplace", "Non-Tobacco Premium": "Non_Tobacco_Premium", "Tobacco Premium": "Tobacco_Premium"})

In [7]:
insurance_df.head()

,Insurance_Company,Rating_Area,Age,Metal_Level,Marketplace,Non_Tobacco_Premium,Tobacco_Premium,Counties
0,"Wellmark, Inc.",Rating Area 2,55,Bronze,Off,753.36,866.36,"Dallas, Jasper, Madison, Marion, Polk, Warren"
1,"Gundersen Health Plan, Inc.",Rating Area 7,60,Bronze,On,863.59,863.59,"Allamakee, Bremer, Butler, Cerro Gordo, Chicka..."
2,Freedom Life Ins. Co.,Rating Area 7,62,Bronze,Off,961.57,1153.89,"Allamakee, Bremer, Butler, Cerro Gordo, Chicka..."
3,"Gundersen Health Plan, Inc.",Rating Area 7,25,Gold,On,472.06,472.06,"Allamakee, Bremer, Butler, Cerro Gordo, Chicka..."
4,Freedom Life Ins. Co.,Rating Area 2,43,Bronze,Off,408.72,490.46,"Dallas, Jasper, Madison, Marion, Polk, Warren"


Wiil now separate the county values in the Rating_Area column into their own column County so that we can work with county level data.


In [8]:
#Making sure there is consistent count across rows so that we can work with the data
insurance_df.groupby(['Rating_Area', 'Counties']).count()

,,Insurance_Company,Age,Metal_Level,Marketplace,Non_Tobacco_Premium,Tobacco_Premium
Rating_Area,Counties,,,,,,
Rating Area 1,"Boone, Calhoun, Carroll, Greene, Grundy, Hamilton, Hardin, Marshall, Poweshiek, Story, Tama, Webster",2024,2024,2024,2024,2024,2024
Rating Area 2,"Dallas, Jasper, Madison, Marion, Polk, Warren",2300,2300,2300,2300,2300,2300
Rating Area 3,"Buena Vista, Cherokee, Clay, Crawford, Dickinson, Ida, Lyon, Monona, O’Brien, Osceola, Palo Alto, Plymouth, Pocahontas, Sac, Sioux, Woodbury",2024,2024,2024,2024,2024,2024
Rating Area 4,"Adair, Adams, Audubon, Cass, Clarke, Decatur, Fremont, Guthrie, Harrison, Mills, Montgomery, Page, Pottawattamie, Ringgold, Shelby, Taylor, Union",2668,2668,2668,2668,2668,2668
Rating Area 5,"Appanoose, Davis, Des Moines, Henry, Jefferson, Keokuk, Lee, Louisa, Lucas, Mahaska, Monroe, Muscatine, Van Buren, Wapello, Washington, Wayne",2254,2254,2254,2254,2254,2254
Rating Area 6,"Benton, Black Hawk, Buchanan, Cedar, Clayton, Clinton, Delaware, Dubuque, Iowa, Jackson, Johnson, Jones, Linn, Scott",3542,3542,3542,3542,3542,3542
Rating Area 7,"Allamakee, Bremer, Butler, Cerro Gordo, Chickasaw, Emmet, Fayette, Floyd, Franklin, Hancock, Howard, Humboldt, Kossuth, Mitchell, Winnebago, Winneshiek, Worth, Wright",2760,2760,2760,2760,2760,2760


In [9]:
insurance_df_by_rating_area = insurance_df.groupby('Rating_Area')
#insurance_df_by_rating_area = insurance_df.groupby(["Rating_Area", "Counties"])

In [10]:
#insurance_df_by_rating_area.head(5)

In [11]:
new_df = pd.DataFrame(insurance_df.Counties.str.split(',').tolist(), index=insurance_df.Rating_Area).stack()
#new_df

In [12]:
new_df = new_df.reset_index([0, 'Rating_Area'])

In [13]:
new_df.columns = ['Rating_Area', 'County']

In [14]:
new_df['n'] = 1

In [15]:
county_rating_area_df = new_df.groupby(["Rating_Area", "County"], as_index=False).count().drop('n', axis=1)

In [16]:
county_rating_area_df['County'] = county_rating_area_df['County'].str.replace(" ", "")
county_rating_area_df['County'] = county_rating_area_df['County'].str.replace("’", "")
county_rating_area_df.head(3)

,Rating_Area,County
0,Rating Area 1,Calhoun
1,Rating Area 1,Carroll
2,Rating Area 1,Greene


Webscraping below to get population data for each county to incorporate into model. 

In [17]:
county_pop_url = ('https://en.wikipedia.org/wiki/List_of_counties_in_Iowa')
county_pop_table = pd.read_html(county_pop_url)  
county_pop_df = county_pop_table[1]
county_pop_df.head(3)

,County,FIPS code[10],County seat[4],#,Established[4][11],Formed from[12],Etymology[11],Map #,Population[13],Area[4],Map
0,Adair County,1,Greenfield,1,"Jan 15, 1851",Cass County,"John Adair, Kentucky Governor",69,7682,"569 sq mi(1,474 km2)",NaN
1,Adams County,3,Corning,2,"Jan 15, 1851",Taylor County,"John Adams, US President",81,4029,"424 sq mi(1,098 km2)",NaN
2,Allamakee County,5,Waukon,3,"Feb 20, 1847",Clayton County,"Disputed, possibly Allan Makee, early trader a...",11,14330,"640 sq mi(1,658 km2)",NaN


In [18]:
#Reading in  county level census data to incorporate a variety of economic and demographic data into predictive model
counties_df = pd.read_csv('ACS_17_5YR_DP03_with_ann.csv', header = 1)
counties_df.head(3)

,Id,Id2,Geography,Estimate; EMPLOYMENT STATUS - Population 16 years and over,Margin of Error; EMPLOYMENT STATUS - Population 16 years and over,Percent; EMPLOYMENT STATUS - Population 16 years and over,Percent Margin of Error; EMPLOYMENT STATUS - Population 16 years and over,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Margin of Error; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force,...,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over,Percent Margin of Error; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over,Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - People in families,Margin of Error; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - People in families,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - People in families,Percent Margin of Error; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - People in families,Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Unrelated individuals 15 years and over,Margin of Error; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Unrelated individuals 15 years and over,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Unrelated individuals 15 years and over,Percent Margin of Error; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Unrelated individuals 15 years and over
0,0500000US19001,19001,"Adair County, Iowa",5810,34,5810,(X),3794,96,65.3,...,9.9,2.8,(X),(X),5.4,2.1,(X),(X),25.6,4.4
1,0500000US19003,19003,"Adams County, Iowa",3087,24,3087,(X),1875,86,60.7,...,7.3,2.9,(X),(X),5.8,2.5,(X),(X),30.8,5.7
2,0500000US19005,19005,"Allamakee County, Iowa",11106,64,11106,(X),7118,188,64.1,...,7.5,2.4,(X),(X),8.8,2.3,(X),(X),22.3,3.9


#### Cannot use the webscraped population data from Wikipedia above since it is for total population, and all other data is for populations at least 16 year and over.

In [19]:
#FILTERING OUT UNNEEDED COLUMNS
counties_df = counties_df[counties_df.columns.drop(list(counties_df.filter(regex='Error')))]
counties_df = counties_df[counties_df.columns.drop(list(counties_df.filter(regex='OCCUPATION')))]
counties_df = counties_df[counties_df.columns.drop(list(counties_df.filter(regex='INDUSTRY')))]
counties_df = counties_df[counties_df.columns.drop(list(counties_df.filter(regex='CLASS OF WORKER')))]
counties_df = counties_df[counties_df.columns.drop(list(counties_df.filter(regex='Id')))]
counties_df = counties_df[counties_df.columns.drop(list(counties_df.filter(regex='Armed')))]
counties_df = counties_df.replace("(X)", np.NaN)
counties_df = counties_df.dropna(axis=1, how='all')


In [20]:
counties_df.shape

(99, 187)

In [21]:
counties_df.head()

,Geography,Estimate; EMPLOYMENT STATUS - Population 16 years and over,Percent; EMPLOYMENT STATUS - Population 16 years and over,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Employed,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Employed,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed,...,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years - Related children of the householder under 18 years,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years - Related children of the householder under 18 years - Related children of the householder under 5 years,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years - Related children of the householder under 18 years - Related children of the householder 5 to 17 years,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 18 to 64 years,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - People in families,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Unrelated individuals 15 years and over
0,"Adair County, Iowa",5810,5810,3794,65.3,3794,65.3,3680,63.3,114,...,9.5,12.1,11.7,24.6,7.5,8.8,8.4,9.9,5.4,25.6
1,"Adams County, Iowa",3087,3087,1875,60.7,1875,60.7,1796,58.2,79,...,11.2,8.6,8.6,11.9,7.3,11.9,13.7,7.3,5.8,30.8
2,"Allamakee County, Iowa",11106,11106,7118,64.1,7118,64.1,6881,62.0,237,...,11.4,15.3,15.2,20.5,13.2,10.2,11.2,7.5,8.8,22.3
3,"Appanoose County, Iowa",10061,10061,5874,58.4,5874,58.4,5502,54.7,372,...,17.1,26.3,25.7,28.9,24.6,14.5,16.3,10.0,13.2,30.3
4,"Audubon County, Iowa",4710,4710,2972,63.1,2962,62.9,2882,61.2,80,...,11.1,15.3,14.6,22.5,11.9,10.1,11.0,7.7,8.6,19.8


In [1]:
#Print out all remaining columns to look at which ones might be useful for predictive model
#for column in counties_df:
#    print(column)

In [585]:
#List of columns from Census data to use for Regression
column_list = ['Geography',
              'Estimate; EMPLOYMENT STATUS - Population 16 years and over', 
               'Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed',
              'Percent; EMPLOYMENT STATUS - Population 16 years and over - Not in labor force',
              'Percent; EMPLOYMENT STATUS - Females 16 years and over',
              'Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)',
              'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)',
              'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars)',
              'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for male full-time, year-round workers (dollars)',
              'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for female full-time, year-round workers (dollars)',
              'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With private health insurance',
              'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With public coverage',
              'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage',
              'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people',
              'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over',
              'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years']
               


In [586]:
shorter_df = counties_df[column_list]

In [587]:
shorter_df['Geography'] = shorter_df['Geography'].str.replace(" County, Iowa", "")

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [588]:
shorter_df.head()

,Geography,Estimate; EMPLOYMENT STATUS - Population 16 years and over,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed,Percent; EMPLOYMENT STATUS - Population 16 years and over - Not in labor force,Percent; EMPLOYMENT STATUS - Females 16 years and over,Estimate; COMMUTING TO WORK - Mean travel time to work (minutes),Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars),Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars),"Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for male full-time, year-round workers (dollars)","Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for female full-time, year-round workers (dollars)",Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With private health insurance,Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With public coverage,Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over,Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years
0,Adair,5810,2.0,34.7,2956,22.3,28861,32169,44421,36288,73.1,38.7,6.4,9.5,9.9,12.1
1,Adams,3087,2.6,39.3,1584,19.6,27022,29727,42583,30139,75.8,38.8,5.8,11.2,7.3,8.6
2,Allamakee,11106,2.1,35.9,5480,24.3,27377,30458,44464,32808,71.6,37.6,8.6,11.4,7.5,15.3
3,Appanoose,10061,3.7,41.6,5184,19.6,25543,28643,41576,30561,63.9,45.3,8.4,17.1,10.0,26.3
4,Audubon,4710,1.7,36.9,2406,19.2,31523,29331,44400,29323,70.3,42.7,6.6,11.1,7.7,15.3


In [589]:
shorter_df.columns

Index(['Geography',
       'Estimate; EMPLOYMENT STATUS - Population 16 years and over',
       'Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed',
       'Percent; EMPLOYMENT STATUS - Population 16 years and over - Not in labor force',
       'Percent; EMPLOYMENT STATUS - Females 16 years and over',
       'Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)',
       'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)',
       'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars)',
       'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for male full-time, year-round workers (dollars)',
       'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for female full-time, year-round workers (dollars)',
       'Percent; HEALTH INSURANCE COVERAGE - Ci

I will now edit all the remaining column names below to make them much more readable and understandable

In [590]:

shorter_df = shorter_df.rename(columns = {"Geography":"County", "Estimate; EMPLOYMENT STATUS - Population 16 years and over":"Popul_16_Over",
                                          'Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed':'Unemployment_Rate',
                                          'Percent; EMPLOYMENT STATUS - Population 16 years and over - Not in labor force':'Not_in_Labor_Force_Pct',
                                          'Percent; EMPLOYMENT STATUS - Females 16 years and over':'Female_Popul_16_Over', 
                                          'Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)':'Avg_Commute_Minutes',
                                          'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)':'Per_capita_income',
                                          'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars)':'Median_Income',
                                          'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for male full-time, year-round workers (dollars)':'Median_Male_Income',
                                          'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for female full-time, year-round workers (dollars)':'Median_Female_Income',
                                          'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With private health insurance': 'Priv_Health_Ins_Pct',
                                          'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With public coverage':'Pub_Health_Ins_Pct',
                                          'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage':'No_Health_Ins_Pct',
                                          'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people':'Poverty_Rate_Total',
                                          'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over':'Poverty_Rate_Over_65',
                                          'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years':'Poverty_Rate_Children'})
                                    

In [591]:
shorter_df.head()
shorter_df['County'] = shorter_df['County'].str.replace(" ", "")
shorter_df['County'] = shorter_df['County'].str.replace("'", "")

In [592]:
county_census_df = pd.merge(county_rating_area_df, shorter_df, 'left', ['County'])
county_census_df.head(3)

,Rating_Area,County,Popul_16_Over,Unemployment_Rate,Not_in_Labor_Force_Pct,Female_Popul_16_Over,Avg_Commute_Minutes,Per_capita_income,Median_Income,Median_Male_Income,Median_Female_Income,Priv_Health_Ins_Pct,Pub_Health_Ins_Pct,No_Health_Ins_Pct,Poverty_Rate_Total,Poverty_Rate_Over_65,Poverty_Rate_Children
0,Rating Area 1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,30671,76.3,40.5,3.6,15.3,9.1,21.3
1,Rating Area 1,Carroll,15879,1.7,31.4,8172,14.0,29191,32365,46455,36187,80.7,33.4,2.5,10.3,8.7,12.8
2,Rating Area 1,Greene,7300,4.1,34.2,3788,19.6,27968,29346,44864,36695,74.7,38.8,5.3,10.0,6.3,11.7


In [593]:
#Merging healthcare plan dataframe with census dataframe
combined_df = pd.merge(county_census_df, insurance_df, 'left', ['Rating_Area'])

In [594]:
combined_df.head()

,Rating_Area,County,Popul_16_Over,Unemployment_Rate,Not_in_Labor_Force_Pct,Female_Popul_16_Over,Avg_Commute_Minutes,Per_capita_income,Median_Income,Median_Male_Income,...,Poverty_Rate_Total,Poverty_Rate_Over_65,Poverty_Rate_Children,Insurance_Company,Age,Metal_Level,Marketplace,Non_Tobacco_Premium,Tobacco_Premium,Counties
0,Rating Area 1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,15.3,9.1,21.3,Freedom Life Ins. Co.,60,Bronze,Off,844.53,1013.44,"Boone, Calhoun, Carroll, Greene, Grundy, Hamil..."
1,Rating Area 1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,15.3,9.1,21.3,Freedom Life Ins. Co.,42,Bronze,Off,412.31,494.77,"Boone, Calhoun, Carroll, Greene, Grundy, Hamil..."
2,Rating Area 1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,15.3,9.1,21.3,Freedom Life Ins. Co.,28,Bronze,Off,338.25,405.90,"Boone, Calhoun, Carroll, Greene, Grundy, Hamil..."
3,Rating Area 1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,15.3,9.1,21.3,Freedom Life Ins. Co.,20,Bronze,Off,197.60,237.12,"Boone, Calhoun, Carroll, Greene, Grundy, Hamil..."
4,Rating Area 1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,15.3,9.1,21.3,Freedom Life Ins. Co.,47,Bronze,Off,486.37,583.64,"Boone, Calhoun, Carroll, Greene, Grundy, Hamil..."


In [595]:
combined_df['Rating_Area'] = combined_df['Rating_Area'].str.replace("Rating Area ", "")
combined_df = combined_df.drop(columns=['Counties'])

In [596]:
combined_df.head()

,Rating_Area,County,Popul_16_Over,Unemployment_Rate,Not_in_Labor_Force_Pct,Female_Popul_16_Over,Avg_Commute_Minutes,Per_capita_income,Median_Income,Median_Male_Income,...,No_Health_Ins_Pct,Poverty_Rate_Total,Poverty_Rate_Over_65,Poverty_Rate_Children,Insurance_Company,Age,Metal_Level,Marketplace,Non_Tobacco_Premium,Tobacco_Premium
0,1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,3.6,15.3,9.1,21.3,Freedom Life Ins. Co.,60,Bronze,Off,844.53,1013.44
1,1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,3.6,15.3,9.1,21.3,Freedom Life Ins. Co.,42,Bronze,Off,412.31,494.77
2,1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,3.6,15.3,9.1,21.3,Freedom Life Ins. Co.,28,Bronze,Off,338.25,405.90
3,1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,3.6,15.3,9.1,21.3,Freedom Life Ins. Co.,20,Bronze,Off,197.60,237.12
4,1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,...,3.6,15.3,9.1,21.3,Freedom Life Ins. Co.,47,Bronze,Off,486.37,583.64


In [597]:
#Convert categorical variables into dummy variables so we can use them in model
combined_df = pd.get_dummies(combined_df, columns=['Rating_Area', 'Marketplace', 'Metal_Level', 'Insurance_Company'], drop_first=True)
combined_df.columns

Index(['County', 'Popul_16_Over', 'Unemployment_Rate',
       'Not_in_Labor_Force_Pct', 'Female_Popul_16_Over', 'Avg_Commute_Minutes',
       'Per_capita_income', 'Median_Income', 'Median_Male_Income',
       'Median_Female_Income', 'Priv_Health_Ins_Pct', 'Pub_Health_Ins_Pct',
       'No_Health_Ins_Pct', 'Poverty_Rate_Total', 'Poverty_Rate_Over_65',
       'Poverty_Rate_Children', 'Age', 'Non_Tobacco_Premium',
       'Tobacco_Premium', 'Rating_Area_2', 'Rating_Area_3', 'Rating_Area_4',
       'Rating_Area_5', 'Rating_Area_6', 'Rating_Area_7', 'Marketplace_On',
       'Metal_Level_Catastrophic', 'Metal_Level_Gold', 'Metal_Level_Platinum',
       'Metal_Level_Silver', 'Insurance_Company_Freedom Life Ins. Co.',
       'Insurance_Company_Gundersen Health Plan, Inc.',
       'Insurance_Company_Medica Insurance Company',
       'Insurance_Company_Wellmark Health Plan of Iowa',
       'Insurance_Company_Wellmark Synergy Health, Inc.',
       'Insurance_Company_Wellmark Value Health Plan, Inc.

In [598]:
#make dummy variables more readable
combined_df = combined_df.rename(columns = {"Insurance_Company_Freedom Life Ins. Co.":"Freedom_Life_Ins_Co", "Insurance_Company_Gundersen Health Plan, Inc.":"Gundersen_Ins_Co",
                                          'Insurance_Company_Medica Insurance Company':'Medica_Ins_Co',
                                          'Insurance_Company_Wellmark Health Plan of Iowa':'Wellmark_Health_Plan_Iowa_Ins_Co',
                                          'Insurance_Company_Wellmark Synergy Health, Inc.':'Wellmark_Synergy_Health_Ins_Co', 
                                          'Insurance_Company_Wellmark Value Health Plan, Inc.':'Wellmark_Value_Health_Plan_Ins_Co',
                                          'Insurance_Company_Wellmark, Inc.':'Wellmark_Ins_Co'})
                                          

In [599]:
combined_df.head(2)

,County,Popul_16_Over,Unemployment_Rate,Not_in_Labor_Force_Pct,Female_Popul_16_Over,Avg_Commute_Minutes,Per_capita_income,Median_Income,Median_Male_Income,Median_Female_Income,...,Metal_Level_Gold,Metal_Level_Platinum,Metal_Level_Silver,Freedom_Life_Ins_Co,Gundersen_Ins_Co,Medica_Ins_Co,Wellmark_Health_Plan_Iowa_Ins_Co,Wellmark_Synergy_Health_Ins_Co,Wellmark_Value_Health_Plan_Ins_Co,Wellmark_Ins_Co
0,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,30671,...,0,0,0,1,0,0,0,0,0,0
1,Calhoun,7933,1.3,43.4,3854,20.4,27370,26657,44283,30671,...,0,0,0,1,0,0,0,0,0,0


In [600]:
combined_df.columns

Index(['County', 'Popul_16_Over', 'Unemployment_Rate',
       'Not_in_Labor_Force_Pct', 'Female_Popul_16_Over', 'Avg_Commute_Minutes',
       'Per_capita_income', 'Median_Income', 'Median_Male_Income',
       'Median_Female_Income', 'Priv_Health_Ins_Pct', 'Pub_Health_Ins_Pct',
       'No_Health_Ins_Pct', 'Poverty_Rate_Total', 'Poverty_Rate_Over_65',
       'Poverty_Rate_Children', 'Age', 'Non_Tobacco_Premium',
       'Tobacco_Premium', 'Rating_Area_2', 'Rating_Area_3', 'Rating_Area_4',
       'Rating_Area_5', 'Rating_Area_6', 'Rating_Area_7', 'Marketplace_On',
       'Metal_Level_Catastrophic', 'Metal_Level_Gold', 'Metal_Level_Platinum',
       'Metal_Level_Silver', 'Freedom_Life_Ins_Co', 'Gundersen_Ins_Co',
       'Medica_Ins_Co', 'Wellmark_Health_Plan_Iowa_Ins_Co',
       'Wellmark_Synergy_Health_Ins_Co', 'Wellmark_Value_Health_Plan_Ins_Co',
       'Wellmark_Ins_Co'],
      dtype='object')

In [601]:
#Save final dataframe to be used for regression models
combined_df.to_csv('combined_df.csv')